In [48]:
%matplotlib inline

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib as mlt

In [49]:
df = pd.read_csv('marathon_raw.csv', sep=';')
df.head(1)

,Name_country,Age_group,Number,Time,Pace_0_5,Pace_35_40,First_half,Second_half
0,"Adola, Guye (ETH)",M30,2.0,02:05:48,02:53,03:12,01:00:48,01:05:00


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name_country  1 non-null      object 
 1   Age_group     1 non-null      object 
 2   Number        1 non-null      float64
 3   Time          1 non-null      object 
 4   Pace_0_5      1 non-null      object 
 5   Pace_35_40    1 non-null      object 
 6   First_half    1 non-null      object 
 7   Second_half   1 non-null      object 
dtypes: float64(1), object(7)
memory usage: 1.8+ KB


In [51]:
df['Country'] = df['Name_country'].str[-4:-1]
df.head(1)

,Name_country,Age_group,Number,Time,Pace_0_5,Pace_35_40,First_half,Second_half,Country
0,"Adola, Guye (ETH)",M30,2.0,02:05:48,02:53,03:12,01:00:48,01:05:00,ETH


In [52]:
df['Gender'] = df['Age_group'].str[0:1]
df.head(1)

,Name_country,Age_group,Number,Time,Pace_0_5,Pace_35_40,First_half,Second_half,Country,Gender
0,"Adola, Guye (ETH)",M30,2.0,02:05:48,02:53,03:12,01:00:48,01:05:00,ETH,M


In [53]:
df['Age'] = df['Age_group'].str[1:]
df.head(1)

,Name_country,Age_group,Number,Time,Pace_0_5,Pace_35_40,First_half,Second_half,Country,Gender,Age
0,"Adola, Guye (ETH)",M30,2.0,02:05:48,02:53,03:12,01:00:48,01:05:00,ETH,M,30


In [54]:
df2 = pd.read_csv('wikipedia-iso-country-codes.csv')
df2.head(2)

,English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX


In [55]:
df2.rename(columns={'Alpha-3 code': 'Country', 'English short name lower case': 'Country_long'}, inplace=True)
df2

,Country_long,Alpha-2 code,Country,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,8,ISO 3166-2:AL
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ
4,American Samoa,AS,ASM,16,ISO 3166-2:AS
...,...,...,...,...,...
241,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF
242,Western Sahara,EH,ESH,732,ISO 3166-2:EH
243,Yemen,YE,YEM,887,ISO 3166-2:YE
244,Zambia,ZM,ZMB,894,ISO 3166-2:ZM


In [56]:
df2.loc[:, ['Country_long','Country']].head()

,Country_long,Country
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM


In [57]:
df_merged = pd.merge(df, 
                      df2.loc[:, ['Country_long','Country']], 
                      on ='Country', 
                      how ='inner')

In [58]:
df_merged

,Name_country,Age_group,Number,Time,Pace_0_5,Pace_35_40,First_half,Second_half,Country,Gender,Age,Country_long
0,"Adola, Guye (ETH)",M30,2.0,02:05:48,02:53,03:12,01:00:48,01:05:00,ETH,M,30,Ethiopia


In [61]:
df_merged[['Name','Country2']] = df_merged.Name_country.apply(
   lambda x: pd.Series(str(x).split("(")))
df_merged.drop('Country2', axis=1, inplace=True)

In [62]:
df_merged

,Name_country,Age_group,Number,Time,Pace_0_5,Pace_35_40,First_half,Second_half,Country,Gender,Age,Country_long,Name
0,"Adola, Guye (ETH)",M30,2.0,02:05:48,02:53,03:12,01:00:48,01:05:00,ETH,M,30,Ethiopia,"Adola, Guye"


In [70]:
times = df_merged['Time']
times.describe()

count            1
unique           1
top       02:05:48
freq             1
Name: Time, dtype: object

In [71]:
# counting minutes from Time

tot_min = []
times = list(times)

for i in times:
    t=i.split(':')
    tot_min.append(int(t[0])*60+int(t[1])*1 +int(t[2])/60)

In [72]:
tot_min

[125.8]

In [76]:
time_firsthalf = df_merged['First_half']
time_secondhalf = df_merged['Second_half']

In [82]:
# counting minutes from Fisrt_half and Second_half

tot_min_firsthalf = []
time_firsthalf = list(time_firsthalf)

for i in time_firsthalf:
    t=i.split(':')
    tot_min_firsthalf.append(int(t[0])*60+int(t[1])*1 +int(t[2])/60)
    

In [87]:
tot_min_firsthalf

[60.8]

In [88]:
tot_min_secondhalf

[65.0]

In [106]:
time_Pace_05 = df_merged['Pace_0_5']
time_Pace_3540 = df_merged['Pace_35_40']

In [107]:
# counting seconds from Pace_0_5 and Pace_35_40

tot_sec_0_5 = []
time_Pace_05 = list(time_Pace_05)

for i in time_Pace_05:
    t=i.split(':')
    tot_sec_0_5.append(int(t[0])*60 +int(t[1])*1)
    
tot_sec_35_40 = []
time_Pace_3540 = list(time_Pace_3540)

for i in time_Pace_3540:
    t=i.split(':')
    tot_sec_35_40.append(int(t[0])*60 +int(t[1])*1)

In [104]:
tot_sec_0_5

[173]

In [108]:
tot_sec_35_40

[192]

In [130]:
Name = df_merged['Name']

In [131]:
times_new = pd.DataFrame(
    {'Time': tot_min,
     'First_half': tot_min_firsthalf,
     'Second_half': tot_min_secondhalf,
     'Pace_0_5': tot_sec_0_5,
     'Pace_35_40':tot_sec_35_40,
     'Name':Name
    })

In [132]:
times_new

,Time,First_half,Second_half,Pace_0_5,Pace_35_40,Name
0,125.8,60.8,65.0,173,192,"Adola, Guye"


In [124]:
times_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Time         1 non-null      float64
 1   First_half   1 non-null      float64
 2   Second_half  1 non-null      float64
 3   Pace_0_5     1 non-null      int64  
 4   Pace_35_40   1 non-null      int64  
dtypes: float64(3), int64(2)
memory usage: 168.0 bytes


In [129]:
times_new['Time'] = int(times_new['Time'])
times_new['First_half'] = int(times_new['First_half'])
times_new['Second_half'] = int(times_new['Second_half'])
df_merged['Number'] = int(df_merged['Number'])

In [133]:
df_joined = pd.merge(df_merged, 
                      times_new, 
                      on ='Name', 
                      how ='inner')

In [134]:
df_joined

,Name_country,Age_group,Number,Time_x,Pace_0_5_x,Pace_35_40_x,First_half_x,Second_half_x,Country,Gender,Age,Country_long,Name,Time_y,First_half_y,Second_half_y,Pace_0_5_y,Pace_35_40_y
0,"Adola, Guye (ETH)",M30,2,02:05:48,02:53,03:12,01:00:48,01:05:00,ETH,M,30,Ethiopia,"Adola, Guye",125.8,60.8,65.0,173,192
